In [ ]:
from __future__ import print_function
from __future__ import division

import sys
import os
qprc_path = os.path.abspath(os.path.join('..'))
if qprc_path not in sys.path:
    sys.path.append(qprc_path)
    
from utils.plots import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.ma as ma

import seaborn as sns
from  kernelfuzzy.fuzzyset import FuzzySet
from kernelfuzzy.fuzzysystem import get_rule_antecedents,plot_membership_fun
from  kernelfuzzy.fuzzification import FuzzyData, NonSingletonFuzzifier
from kernelfuzzy.kernels import gram_matrix_KBF_kernel,KBFkernel,NonSingletonKernel
from sklearn.datasets.samples_generator import make_classification
from sklearn.datasets import make_moons, make_circles,make_blobs,load_digits
from sklearn.svm import SVC,NuSVC
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from kernelfuzzy.kernels import nonsingleton_gaussian_kernel, gram_matrix_nonsingleton_gaussian_kernel
from sklearn.metrics import accuracy_score
import skfuzzy as fuzz # for FCM
import pickle
from sklearn.neighbors import KNeighborsClassifier

from fylearn.nfpc import FuzzyPatternClassifier
from fylearn.garules import MultimodalEvolutionaryClassifier
from fylearn.fpt import FuzzyPatternTreeTopDownClassifier

def NSFS_predict(model, X,X_test, option=0):
    #getting best cv parameters
    gamma=clf.best_params_['NSKernel__param']
    std_proportion=clf.best_params_['Fuzzifier__std_proportion']
    best_model=model.best_estimator_['svm']
    
    if option==0:
        ############################
        # approach 1:  not embedding the gamma
        ############################
        print("First approach")
        fuzzy_data=NonSingletonFuzzifier( std_proportion=std_proportion, constant_std=True ).transform(X)
        fuzzy_data_test=NonSingletonFuzzifier( std_proportion=std_proportion, constant_std=True ).transform(X_test)
        #KBF/FBF expansion
        K=gram_matrix_KBF_kernel(fuzzy_data_test,fuzzy_data, gamma) 
        y_pred=(K[:,best_model.support_]@best_model.dual_coef_.T+best_model.intercept_).flatten()
        return K, y_pred
    
    if option==1:
        ############################
        # approach 2: embedding the gamma
        ############################
        print("Second approach")
        #prediction with a FBF as KBF constructed using the nonsingleton kernel
        #nonsingleton fuzzyfication
        fuzzy_data=NonSingletonFuzzifier( std_proportion=std_proportion*np.sqrt(1/gamma), constant_std=True ).transform(X)
        fuzzy_data_test=NonSingletonFuzzifier( std_proportion=std_proportion*np.sqrt(1/gamma), constant_std=True ).transform(X_test)
        #KBF/FBF expansion
        K=gram_matrix_KBF_kernel(fuzzy_data_test,fuzzy_data, 1) # gamma=1 because we actually embedded the gamma in the fuzzification and in the rules
        y_pred=(K[:,best_model.support_]@best_model.dual_coef_.T+best_model.intercept_).flatten()
        return K, y_pred
    
    if option==2:
        ############################
        # approach 3: embedding the gamma only in the input and using the rules
        ############################
        print("Third approach I")
        fuzzy_data=NonSingletonFuzzifier( std_proportion=std_proportion, constant_std=True ).transform(X)
        rules_antecedents=get_rule_antecedents(best_model, fuzzy_data, gamma)
                
        fuzzy_data_test=NonSingletonFuzzifier( std_proportion=std_proportion*np.sqrt(1/gamma), constant_std=True ).transform(X_test)
        K=gram_matrix_KBF_kernel(fuzzy_data_test,rules_antecedents, 1) # gamma=1 because we actually embedded the gamma in the fuzzification and in the rules
        # equivalent to: K=gram_matrix_KBF_kernel(fuzzy_data_test,fuzzy_data[clf.support_], 1) # gamma=1 because we actually embedded the gamma in the fuzzification and in the rules
        #print("kernel : {}", K)
        y_pred=(K@best_model.dual_coef_.T+best_model.intercept_).flatten()
        return K, y_pred
    
    if option==3:
        ############################
        # approach 3: embedding the gamma only in the input and using the rules
        ############################
        print("Third approach II")       
        rule_antecedents=NonSingletonFuzzifier( std_proportion=std_proportion*np.sqrt(1/gamma), constant_std=True ).transform(X[best_model.support_, :])
        fuzzy_data_test=NonSingletonFuzzifier( std_proportion=std_proportion*np.sqrt(1/gamma), constant_std=True ).transform(X_test)
        K=gram_matrix_KBF_kernel(fuzzy_data_test,rule_antecedents, 1) # gamma=1 because we actually embedded the gamma in the fuzzification and in the rules
        # equivalent to: K=gram_matrix_KBF_kernel(fuzzy_data_test,fuzzy_data[clf.support_], 1) # gamma=1 because we actually embedded the gamma in the fuzzification and in the rules
        #print("kernel : {}", K)
        y_pred=(K@best_model.dual_coef_.T+best_model.intercept_).flatten()
        return K, y_pred
        
    if option==4:
        ############################
        # prediction of a SVM and the non-singleton kernel
        ############################
        print("SVM + nonsingleton kernel")
        fuzzy_data=NonSingletonFuzzifier( std_proportion=std_proportion, constant_std=True ).transform(X)
        fuzzy_data_test=NonSingletonFuzzifier( std_proportion=std_proportion, constant_std=True ).transform(X_test)
        #KBF/FBF expansion
        K=gram_matrix_nonsingleton_gaussian_kernel(fuzzy_data_test,fuzzy_data, gamma) 
        y_pred=best_model.predict(K)
        return K, y_pred


This notebook estimates the accuracy of NSFS trained with kernels as a function of data dimension

##  Dataset


In [ ]:
#classifier
pipe = Pipeline([
    ('Fuzzifier', NonSingletonFuzzifier(constant_std=True)),
    ('NSKernel', NonSingletonKernel()),
    #('svm', NuSVC(),
    ('svm', SVC())])

pipe_KBF = Pipeline([
    ('Fuzzifier', NonSingletonFuzzifier(constant_std=True)),
    ('Kernel', KBFkernel()),
    ('svm', SVC())])

#noisy train - noisy test
#data_dimension=[4,8,16,32,64,128,256,512,1024]
data_dimension=range(10,500,50)
data_dimension=range(2,20,1)

fuzzy_classifiers = (FuzzyPatternClassifier(),
                     MultimodalEvolutionaryClassifier(),
                     FuzzyPatternTreeTopDownClassifier())


list_options_predict=[0,1,2,3,4]
acc=[]
list_results=[]
for d in data_dimension:
    print("Dimension : {}".format(d))
    #X, y = make_classification(n_samples=1000, n_features=d, n_redundant=0, n_informative=int(90/100*d), random_state=1, n_clusters_per_class=1)
    #X_test, y_test = make_classification(n_features=d, n_redundant=0, n_informative=int(90/100*d), random_state=1, n_clusters_per_class=1)
    
    X_, y_ = make_classification(n_samples=100, n_features=d,
                                    n_informative=d, n_redundant=0)

    train_samples = 50  # Samples used for training the models

    X = X_[:train_samples]
    y = y_[:train_samples]

    X_test = X_[train_samples:]
    y_test = y_[train_samples:]
     
    #grid search over KBF kernel parameter   
    cv_params = dict([
        ('Fuzzifier__std_proportion',np.arange(0.01,np.std(X),0.1)),
        ('NSKernel__param', 2.0**np.arange(-20,1)),
        ('svm__kernel', ['precomputed']),
        #('svm__nu', np.arange(0.05,0.1,0.05))
        ('svm__C', 2.0**np.arange(-15,15))
    ])    
    #training
    clf = RandomizedSearchCV(pipe, cv_params, cv=5, verbose=1, n_jobs=-1,n_iter=30)
    clf.fit(X, y)
    
    #predict
    #for opt in list_options_predict:
    #    K,y_pred=NSFS_predict(clf, X,X_test, option=opt)
    #    if opt==0 or opt==1 or opt==4:
    #        print("acc K", accuracy_score(y_test, clf.best_estimator_['svm'].predict(K)))
    #    print("acc y)", accuracy_score(y_test, sign_fun(y_pred)))
    
     
    results={'dimension':d}
    K,y_pred=NSFS_predict(clf, X,X_test, option=0)
    results.update({'NSFS' : accuracy_score(y_test, clf.best_estimator_['svm'].predict(K))} )
    
    #denominator using only the sum of support vectors
    K,y_pred=NSFS_predict(clf, X,X_test, option=3)
    results.update({'NSFS_den' : accuracy_score(y_test, sign_fun(y_pred))})
    
    #SVM + nonsingleton kernel
    results.update({'SVM_NS' : accuracy_score(y_test, clf.predict(X_test))})
        
 #   print('Model parameters : {}'.format(clf.best_params_))
 #   print('support vectors: {}'.format( clf.best_estimator_['svm'].support_))
    
    #getting best cv parameters
    gamma=clf.best_params_['NSKernel__param']
    std_proportion=clf.best_params_['Fuzzifier__std_proportion']
    #print('best parameters gamma, std_proportion, nro_sv ',gamma,std_proportion,clf.best_estimator_['svm'].n_support_)
    best_model=clf.best_estimator_['svm']
    
    results.update({'gamma' : gamma,'std_proportion':std_proportion,'nro_sv':clf.best_estimator_['svm'].n_support_})
    
    #########################
    # SVM + KBF kernel
    #########################
    cv_params = dict([
        ('Fuzzifier__std_proportion',np.arange(0.01,np.std(X),0.1)),
        ('Kernel__param', 2.0**np.arange(-20,-1)),
        ('svm__kernel', ['precomputed']),
        ('svm__C', 2.0**np.arange(-15,15))])
    
    clf = RandomizedSearchCV(pipe_KBF, cv_params, cv=5, verbose=1, n_jobs=-1,n_iter=30)
    clf.fit(X, y)
    results.update({'SVM_KBF' : accuracy_score(y_test, clf.predict(X_test))})
    results.update({'gamma_KBF' : gamma,'std_proportion_KBF':std_proportion,'nro_sv_KBF':clf.best_estimator_['svm'].n_support_})

       
    
    #a SVM with RBF kernel
    # A C-SVM with Gaussian RBF kernel with cross-validation
    #########################
    #print("SVM + Gaussian kernel + cross validation")
    pipe_SVM = Pipeline([('svm', SVC())])

    #grid search over KBF kernel parameter   
    cv_params_SVM = dict([
        ('svm__gamma', 2.0**np.arange(-20,20)),
        ('svm__C', 2.0**np.arange(-15,15)),
    ])
    model_SVM = RandomizedSearchCV(pipe_SVM, cv_params_SVM, cv=5, verbose=1, n_jobs=1,n_iter=30)
    model_SVM.fit(X, y)
    #print("svm_score :",model_SVM.score(X_test,y_test))
    
    #KNN
    pipe_KNN = Pipeline([('knn', KNeighborsClassifier())])

    #grid search over KBF kernel parameter   
    cv_params_KNN = dict([
        ('knn__n_neighbors', [2,4,8,16,32])
    ])
    
    neigh = RandomizedSearchCV(pipe_KNN, cv_params_KNN, cv=5, verbose=1, n_jobs=1,n_iter=30)
    neigh.fit(X, y)
    #print("knn_score :",neigh.score(X_test,y_test))
    results.update({'svmRBF' : model_SVM.score(X_test,y_test),
                    'knn':neigh.score(X_test,y_test)})
    
    #fuzzy classifiers
    for fc in fuzzy_classifiers:
        results.update({type(fc).__name__ : accuracy_score(y_test, fc.fit(X, y).predict(X_test))})
        
        
    list_results.append(results)


df_results=pd.DataFrame(list_results)
print(df_results)


Dimension : 2
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


First approach
Third approach II
Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [ ]:
#df_results.to_csv('df_dimension10_500.csv')
#df_results=pd.read_csv('df_dimension10_500.csv')
df_results.to_csv('df_dimension2_20.csv')
df_results=pd.read_csv('df_dimension2_20.csv')

df_results=df_results.rename(columns={"NSFS": "NSFS_NS_DEN_AFTER", 
                           "SVM_KBF": "NSFS_SVM_KBF",
                           "SVM_NS": "NSFS_SVM_NS"})

ax = plt.gca()

#NSFS using NS kernel and adding the denominator "after" training
df_results.plot(kind='line',x='dimension',y='NSFS_NS_DEN_AFTER',color='red',style='.-',ax=ax)

#NSFS using NS kernel and adding the denominator using only the sv for that "after" training (the results seem the same as using all the training samples for estimating the denominator)
#df_results.plot(kind='line',x='dimension',y='NSFS_den',color='blue',  ax=ax)

#NSFS using KNF kernel direct connection between NSFS and KM
df_results.plot(kind='line',x='dimension',y='NSFS_SVM_KBF', color='blue',style='.-', ax=ax)

#equivalent to a NSFS without denominator
df_results.plot(kind='line',x='dimension',y='NSFS_SVM_NS',  style='.-',ax=ax)

#df_results.plot(kind='line',x='dimension',y='knn',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='svmRBF',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='FuzzyPatternClassifier',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='MultimodalEvolutionaryClassifier',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='FuzzyPatternTreeTopDownClassifier',  ax=ax)
ax.legend(bbox_to_anchor=(0.5, 0.7))

plt.show()

#df_results.to_csv('df.csv')

#######

ax = plt.gca()

#NSFS using KNF kernel direct connection between NSFS and KM
df_results.plot(kind='line',x='dimension',y='NSFS_SVM_KBF', color='blue', style='.-',ax=ax)


df_results.plot(kind='line',x='dimension',y='knn', style='.-', ax=ax)
df_results.plot(kind='line',x='dimension',y='svmRBF',  style='.-',ax=ax)
#df_results.plot(kind='line',x='dimension',y='FuzzyPatternClassifier',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='MultimodalEvolutionaryClassifier',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='FuzzyPatternTreeTopDownClassifier',  ax=ax)
ax.legend(bbox_to_anchor=(0.5, 0.7))

plt.show()

#######

ax = plt.gca()

#NSFS using KNF kernel direct connection between NSFS and KM
df_results.plot(kind='line',x='dimension',y='NSFS_SVM_KBF', color='blue',style='.-', ax=ax)


#df_results.plot(kind='line',x='dimension',y='knn',  ax=ax)
#df_results.plot(kind='line',x='dimension',y='svmRBF',  ax=ax)
df_results.plot(kind='line',x='dimension',y='FuzzyPatternClassifier', style='.-', ax=ax)
df_results.plot(kind='line',x='dimension',y='MultimodalEvolutionaryClassifier', style='.-', ax=ax)
df_results.plot(kind='line',x='dimension',y='FuzzyPatternTreeTopDownClassifier', style='.-', ax=ax)
ax.legend(bbox_to_anchor=(0.5, 1.1))

plt.show()

#use this
#df = pd.DataFrame(np.random.randn(50, 4), 
#                  index=pd.date_range('1/1/2000', periods=50), columns=list('ABCD'))
#df.plot(style='.-')
#df.plot(style=['+-','o-','.--','s:'])
#df.plot(style='.-', markevery=5)
#df.plot(linestyle='-', markevery=100, marker='o', markerfacecolor='black')

